In [1]:
import sys
sys.path.append("/users/ziyzhang/topology-research/nexullance/IT_boost/build")
from Nexullance_IT_cpp import Nexullance_IT_interface

sys.path.append("/users/ziyzhang/topology-research")
import globals as gl
import topologies.RRG as RRG
import numpy as np

In [2]:
V = 16
D = 5
EPR = (D+1)//2
_network = RRG.RRGtopo(V, D)
ASP, _ = _network.calculate_all_shortest_paths()
ECMP_ASP = gl.ECMP(ASP)
arcs = _network.generate_graph_arcs()

Cap_remote = 10 #GBps
Cap_local = 10 #GBps

M_EPs = gl.generate_uniform_traffic_pattern(V, EPR)
remote_link_flows, local_link_flows = _network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, M_EPs)
max_remote_link_load = np.max(remote_link_flows)/Cap_remote
max_local_link_load = np.max(local_link_flows)/Cap_local
# adapt the traffic scaling factor to 10x saturation
traffic_scaling = 10.0/max(max_local_link_load, max_remote_link_load)
M_EPs = traffic_scaling * M_EPs
remote_link_flows, local_link_flows = _network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, M_EPs)
M_R = gl.convert_M_EPs_to_M_R(M_EPs, V, EPR)

In [3]:
nexu_it = Nexullance_IT_interface(V, arcs, M_R, False)
# nexu_it.set_parameters(0.1, 7.0)
nexu_it.run()
nexu_it.get_max_link_load()

7.840926647186279

In [4]:
nexu_it_2 = Nexullance_IT_interface(V, arcs, M_R, False)
nexu_it_2.set_parameters(1.0, 0.1)
nexu_it_2.run()
nexu_it_2.get_max_link_load()

8.575346946716309

In [5]:
nexu_it_2.get_routing_table()

{(0, 1): [([0, 3, 6, 1], 0.05357152223587036),
  ([0, 3, 6, 1], 0.125),
  ([0, 13, 6, 1], 0.125),
  ([0, 3, 14, 1], 0.125),
  ([0, 10, 14, 1], 0.125),
  ([0, 13, 5, 1], 0.125),
  ([0, 11, 5, 1], 0.125),
  ([0, 13, 4, 1], 0.07142847776412964),
  ([0, 11, 4, 1], 0.125)],
 (0, 2): [([0, 10, 2], 1.0)],
 (0, 3): [([0, 3], 1.0)],
 (0, 4): [([0, 13, 4], 0.011904699727892876),
  ([0, 11, 4], 0.5),
  ([0, 11, 4], 0.48809531331062317)],
 (0, 5): [([0, 13, 5], 0.5), ([0, 11, 5], 0.5)],
 (0, 6): [([0, 3, 6], 0.13095244765281677),
  ([0, 3, 6], 0.13095244765281677),
  ([0, 3, 6], 0.13095244765281677),
  ([0, 3, 6], 0.5),
  ([0, 13, 6], 0.10714265704154968)],
 (0, 7): [([0, 12, 8, 7], 0.1428571492433548),
  ([0, 12, 8, 7], 0.1428571492433548),
  ([0, 12, 15, 7], 0.1428571492433548),
  ([0, 10, 15, 7], 0.1428571492433548),
  ([0, 11, 5, 7], 0.1428571492433548),
  ([0, 12, 8, 7], 0.1428571492433548),
  ([0, 11, 8, 7], 0.1428571492433548)],
 (0, 8): [([0, 12, 8], 0.25), ([0, 12, 8], 0.5), ([0, 11, 8], 

In [6]:
nexu_it_2.get_num_attempts_step_2()

138